Importando Librerias

In [45]:
import polars as pl
import os
import glob
import re


Añadiendo los directorios de trabajo a variables

In [ ]:
directorio_actual = os.getcwd()
directorio_generos = os.path.join(directorio_actual, 'generos')
directorio_con_ranking = os.path.join(directorio_actual, 'generos_con_rankings')

Obtengo los Path de todos Los Archivos que se encuentran en la carpeta `data` es decir los `csv` que guardan los titulos de los distintos generos

In [13]:
path_archivos_generos = []
for archivo in glob.glob(os.path.join(directorio_generos, '*.csv')):
    path_archivos_generos.append(archivo)

Creo los Paths para cuando los generos ya tengan añadido la columna de ranking

In [14]:
path_archivos_modificados = []
for path in path_archivos_generos:
    path_modificado = path.replace('generos', 'generos_con_rankings')
    path_archivos_modificados.append(path_modificado)

***
Ciclo que Añade los Rankings a todos los Titulos de Todos los Generos

In [43]:
df = pl.read_csv('data/rank_steam_actual_scraping.csv')
for index in range(len(path_archivos_generos)):
    df_genero = pl.read_csv(path_archivos_generos[index])
    df_genero = df_genero.with_columns(ranking = pl.lit(0))
    for i in range(len(df_genero)):
        titulo = df_genero[i,0]
        mask = df['titulo'] == titulo
        try:
            ranking = df.filter(mask)['ranking'][0]
        except:
            ranking = 0
        df_genero[i,1] = ranking
    
    df_genero.write_csv(path_archivos_modificados[index])

***
Obtengo a los Generos Estudiados en Una Varaible

In [48]:
generos = []
patron = r'\\([a-zA-Z_]+)\.csv$'
for i in range(len(path_archivos_generos)):
    coincidencias = re.search(patron, path_archivos_generos[i])
    genero = coincidencias.group(1)
    generos.append(genero)

***
Añado una Columna por cada Genero al Dataframe Principal

In [62]:
for genero in generos:
    df = df.with_columns(pl.lit(0).alias(genero))

Necesito los Indices de las columnas que representan a los generos para posteriores asignaciones... Tener en cuenta que la primera columna de Genero ('accion') es la columna 13 y son 27 generos en total

In [110]:
index_generos = [13]
for i in range(26):
    index_generos.append(index_generos[i]+1)

***
Marco con 1 las columnas nuevas (las de Genero creadas en el paso anterior) a las que pertenezca cada titulo. Por ejemplo cuando el Titulo sea "Counter-Strike 2" debera marcarse con 1 la columna 'accion', 'disparo', etc y las columnas que representen los generos a los que pertenezca dicho titulo 

In [120]:
for index in range(len(path_archivos_generos)):
    df_genero = pl.read_csv(path_archivos_generos[index])
    for i in range(len(df_genero)):
        titulo = df_genero[i,0]
        if titulo in df['titulo']:
            mask = df['titulo'] == titulo
            index_row_df = df.filter(mask)['ranking'][0] - 1
            df[index_row_df,index_generos[index]] = 1

In [122]:
df

ranking,titulo,critica_usuarios,aprobacion,cantidad_reviews,windows,mac,linux,fecha_lanzamiento,precio_steam,critica_categorica,fecha_ingles,fecha,accion,animacion_modelado,aventura,carreras,contenido_sexual,deportes,desarrollo_juegos,desnudos,diseno_ilustracion,disparos,documentales,edicion_fotografica,educacion,estrategia,formacion_software,free_to_play,indie,multijugador_masivo,primera_persona,produccion_audio,publicacion_web,rol,sangriento,simulacion,tutoriales,utilidades,violentos
i64,str,str,i64,i64,i64,i64,i64,str,str,i64,str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
1,"""Counter-Strike 2""","""Muy positivas""",87,8160723,1,1,0,"""21 AGO 2012""","""Gratuito""",2,"""21 AUG 2012""","""2012-08-21""",1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
2,"""PUBG: BATTLEGROUNDS""","""Variadas""",58,2397033,1,0,0,"""21 DIC 2017""","""Gratuito""",5,"""21 DEC 2017""","""2017-12-21""",1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0
3,"""The First Descendant""","""Variadas""",54,63966,1,0,0,"""30 JUN 2024""","""Gratuito""",5,"""30 JUN 2024""","""2024-06-30""",1,1,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1
4,"""Dota 2""","""Muy positivas""",81,2278393,1,1,1,"""9 JUL 2013""","""Gratuito""",2,"""9 JUL 2013""","""2013-07-09""",1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0
5,"""Steam Deck""","""Sin Dato""",0,0,1,1,1,"""""","""Sin Dato""",10,"""""",null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
9133,"""Port Royale 2""","""Mayormente positivas""",70,549,1,0,0,"""16 JUN 2009""","""$3.59""",3,"""16 JUN 2009""","""2009-06-16""",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
9134,"""""","""Muy positivas""",91,1504,1,0,0,"""""","""""",2,"""""",null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9135,"""Banda sonora original de Songs…","""Positivas""",10,18,0,0,0,"""10 MAY 2022""","""$5.79""",4,"""10 MAY 2022""","""2022-05-10""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [123]:
df.write_csv('data/julio_2024_rank_steam_con_generos.csv')

In [ ]:
mask = df['critica_categorica'] == 8
df.filter(mask)

***
Extrayendo la Data del dashboard

In [1]:
import polars as pl
import duckdb

In [2]:
df = pl.read_csv('data/julio_2024_rank_steam_con_generos.csv')

In [3]:
df.head(1)

ranking,titulo,critica_usuarios,aprobacion,cantidad_reviews,windows,mac,linux,fecha_lanzamiento,precio_steam,critica_categorica,fecha_ingles,fecha,accion,animacion_modelado,aventura,carreras,contenido_sexual,deportes,desarrollo_juegos,desnudos,diseno_ilustracion,disparos,documentales,edicion_fotografica,educacion,estrategia,formacion_software,free_to_play,indie,multijugador_masivo,primera_persona,produccion_audio,publicacion_web,rol,sangriento,simulacion,tutoriales,utilidades,violentos
i64,str,str,i64,i64,i64,i64,i64,str,str,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
1,"""Counter-Strike 2""","""Muy positivas""",87,8160723,1,1,0,"""21 AGO 2012""","""Gratuito""",2,"""21 AUG 2012""","""2012-08-21""",1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0


Obtengo las Distintas Criticas que hay

In [4]:
tipos_criticas = df.filter(df['critica_usuarios'] != 'Sin Dato')[:,2].unique()
tipos_criticas
lista_criticas = [tipos_criticas[x] for x in range(len(tipos_criticas))]
del tipos_criticas
lista_criticas

['Extremadamente negativas',
 'Negativas',
 'Mayormente positivas',
 'Extremadamente positivas',
 'Muy positivas',
 'Positivas',
 'Muy negativas',
 'Mayormente negativas',
 'Variadas']

En este ciclo Sumarizo los distintos Generos y dentro de Cada Genero, sumarizo las plataformas y las distintas criticas

In [5]:
generos = []
for j in range(13,len(df.columns),1):
    diccionario = {}
    genero = df[:,j].name
    diccionario['genero'] = df[:,j].name
    diccionario['cantidad'] = df[:,j].sum()
    
    df_temporal = df.filter(df[genero] == 1)
    diccionario['windows'] = len(df_temporal.filter(df_temporal['windows'] == 1))
    diccionario['mac'] = len(df_temporal.filter(df_temporal['mac'] == 1))
    diccionario['linux'] = len(df_temporal.filter(df_temporal['linux'] == 1).sum())  

    for critica in lista_criticas:
        diccionario[critica] = len(df_temporal.filter(df_temporal['critica_usuarios'] == critica))
    
    generos.append(diccionario)


Creo el Dataframe y llevo los Generos a Letra Capital

In [6]:
df_generos = pl.DataFrame(generos)
df_generos = df_generos.with_columns(
    genero = pl.col('genero').str.replace("_", " ").str.to_titlecase()
)

df_generos

genero,cantidad,windows,mac,linux,Extremadamente negativas,Negativas,Mayormente positivas,Extremadamente positivas,Muy positivas,Positivas,Muy negativas,Mayormente negativas,Variadas
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""Accion""",3871,3851,559,1,5,1,670,329,1833,184,10,65,560
"""Animacion Modelado""",44,44,9,1,0,0,10,6,16,2,0,0,4
"""Aventura""",3284,3261,477,1,2,1,546,383,1608,108,10,50,449
"""Carreras""",339,338,34,1,2,0,45,15,162,27,2,7,39
"""Contenido Sexual""",493,491,47,1,1,0,108,52,251,13,2,3,53
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Sangriento""",872,872,127,1,2,0,154,90,359,41,3,29,143
"""Simulacion""",2905,2895,648,1,2,1,544,180,1262,181,8,63,512
"""Tutoriales""",87,87,12,1,1,0,18,6,41,7,0,0,9


Creo el Dataframe que totaliza las Plataformas

In [72]:
cantidad_plataformas = [df[:,x].sum() for x in range(5,8,1)]
nombres_plataformas = [df[:,x].name for x in range(5,8,1)]
diccionario = {
    "Plataforma": nombres_plataformas,
    "Cantidad": cantidad_plataformas
}

df_plataformas = pl.DataFrame(diccionario)

df_plataformas = df_plataformas.with_columns((pl.col('Cantidad')/ 9137).alias("Porcentaje del total"))

df_plataformas

Plataforma,Cantidad,Porcentaje del total
str,i64,f64
"""windows""",9017,0.986867
"""mac""",1645,0.180037
"""linux""",2420,0.264857


In [77]:
df_generos.sort('cantidad', descending=True).head(10)

In [91]:
df_generos = df_generos.with_columns(
    (pl.col('Extremadamente negativas') + pl.col('Negativas') + 
     pl.col('Muy negativas') + pl.col('Mayormente negativas')).alias("Suma Negativa")
)

df_generos = df_generos.with_columns(
    (pl.col('Extremadamente positivas') + pl.col('Positivas') + 
     pl.col('Muy positivas') + pl.col('Mayormente positivas')).alias("Suma Positiva")
)

In [94]:
df_generos.write_csv('data/2024_steam_titulos_generos.csv')


In [93]:
df_plataformas

Plataforma,Cantidad,Porcentaje del total
str,i64,f64
"""windows""",9017,0.986867
"""mac""",1645,0.180037
"""linux""",2420,0.264857


***
Acomodo los nombres de las Columnas en el df para que coincidan con los nombres de las columnas del df_genero 

In [7]:
columnas = df.columns

In [13]:
columns = []
for columna in columnas:
    columns.append(columna.replace("_"," ").title())

columns

['Ranking',
 'Titulo',
 'Critica Usuarios',
 'Aprobacion',
 'Cantidad Reviews',
 'Windows',
 'Mac',
 'Linux',
 'Fecha Lanzamiento',
 'Precio Steam',
 'Critica Categorica',
 'Fecha Ingles',
 'Fecha',
 'Accion',
 'Animacion Modelado',
 'Aventura',
 'Carreras',
 'Contenido Sexual',
 'Deportes',
 'Desarrollo Juegos',
 'Desnudos',
 'Diseno Ilustracion',
 'Disparos',
 'Documentales',
 'Edicion Fotografica',
 'Educacion',
 'Estrategia',
 'Formacion Software',
 'Free To Play',
 'Indie',
 'Multijugador Masivo',
 'Primera Persona',
 'Produccion Audio',
 'Publicacion Web',
 'Rol',
 'Sangriento',
 'Simulacion',
 'Tutoriales',
 'Utilidades',
 'Violentos']

In [14]:
df.columns = columns
df.head(1)

Ranking,Titulo,Critica Usuarios,Aprobacion,Cantidad Reviews,Windows,Mac,Linux,Fecha Lanzamiento,Precio Steam,Critica Categorica,Fecha Ingles,Fecha,Accion,Animacion Modelado,Aventura,Carreras,Contenido Sexual,Deportes,Desarrollo Juegos,Desnudos,Diseno Ilustracion,Disparos,Documentales,Edicion Fotografica,Educacion,Estrategia,Formacion Software,Free To Play,Indie,Multijugador Masivo,Primera Persona,Produccion Audio,Publicacion Web,Rol,Sangriento,Simulacion,Tutoriales,Utilidades,Violentos
i64,str,str,i64,i64,i64,i64,i64,str,str,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
1,"""Counter-Strike 2""","""Muy positivas""",87,8160723,1,1,0,"""21 AGO 2012""","""Gratuito""",2,"""21 AUG 2012""","""2012-08-21""",1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0


In [15]:
df.write_csv('data/julio_2024_rank_steam_con_generos.csv')